In [ ]:
from tda.experiments.lid.lid_binary import *

In [ ]:
config = Config(
    epochs=50,
    dataset="MNIST",
    architecture=mnist_lenet.name,
    train_noise=0.0,
    dataset_size=10,
    attack_type="FGSM",
    noise=0.0,
    num_iter=1,
    batch_size=100,
    number_of_nn=20
)

In [ ]:
all_aucs, all_coefs = run_experiment(config)

In [ ]:
all_aucs